<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Major Neural Network Architectures Challenge
## *Data Science Unit 4 Sprint 3 Challenge*

In this sprint challenge, you'll explore some of the cutting edge of data science. This week we studied several famous neural network architectures: 
recurrent neural networks (RNNs), long short-term memory (LSTMs), convolutional neural networks (CNNs), and Generative Adverserial Networks (GANs). In this sprint challenge, you will revisit these models. Remember, we are testing your knowledge of these architectures, not your ability to fit a model with high accuracy. 

__*Caution:*__  These approaches can be pretty heavy computationally. All problems were designed so that you should be able to achieve results within at most 5-10 minutes of runtime on Colab or a comparable environment. If something is running longer, double check your approach!

## Challenge Objectives
*You should be able to:*
* <a href="#p1">Part 1</a>: Train a RNN classification model.
* <a href="#p2">Part 2</a>: Utilize a pre-trained CNN for objective detection.
* <a href="#p3">Part 3</a>: Describe the components of an autoencoder.
* <a href="#p4">Part 4</a>: Describe yourself as a data scientist and elucidate your vision of AI.

<a id="p1"></a>
## Part 1 - RNNs

Use an RNN/LSTM to fit a multi-class classification model on reuters news articles to distinguish topics of articles. The data is already encoded properly for use in an RNN model. 

Your Tasks: 
- Use Keras to fit a predictive model, classifying news articles into topics. 
- Report your overall score and accuracy

For reference, the [Keras IMDB sentiment classification example](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py) will be useful, as well the RNN code we used in class.

__*Note:*__  Focus on getting a running model, not on maximizing accuracy with extreme data size or epoch numbers. Only revisit and push accuracy if you get everything else done!

In [1]:
# Import libraries.
import numpy as np
import pandas as pd
import scipy.stats

from sklearn.metrics import accuracy_score

from tensorflow.keras.datasets import reuters
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM

In [2]:
# Load news article dataset.
(X_train, y_train), (X_test, y_test) = reuters.load_data(num_words=None,
                                                         skip_top=0,
                                                         maxlen=None,
                                                         test_split=0.2,
                                                         seed=723812,
                                                         start_char=1,
                                                         oov_char=2,
                                                         index_from=3)

In [3]:
# Print encoding examples.
word_index = reuters.get_word_index(path="reuters_word_index.json")

print(f"Iran is encoded as {word_index['iran']} in the data.")
print(f"London is encoded as {word_index['london']} in the data.")
print("Words are encoded as numbers in our dataset.")

Iran is encoded as 779 in the data.
London is encoded as 544 in the data.
Words are encoded as numbers in our dataset.


In [4]:
# Define topics.
topic_indexes = pd.DataFrame.from_dict(data={'copper': 6, 
                                             'livestock': 28, 
                                             'gold': 25, 
                                             'money-fx': 19, 
                                             'ipi': 30, 
                                             'trade': 11, 
                                             'cocoa': 0, 
                                             'iron-steel': 31, 
                                             'reserves': 12, 
                                             'tin': 26, 
                                             'zinc': 37, 
                                             'jobs': 34, 
                                             'ship': 13, 
                                             'cotton': 14, 
                                             'alum': 23, 
                                             'strategic-metal': 27, 
                                             'lead': 45, 
                                             'housing': 7, 
                                             'meal-feed': 22, 
                                             'gnp': 21, 
                                             'sugar': 10, 
                                             'rubber': 32, 
                                             'dlr': 40,
                                             'veg-oil': 2,  
                                             'interest': 20, 
                                             'crude': 16, 
                                             'coffee': 9, 
                                             'wheat': 5, 
                                             'carcass': 15, 
                                             'lei': 35, 
                                             'gas': 41, 
                                             'nat-gas': 17, 
                                             'oilseed': 24, 
                                             'orange': 38, 
                                             'heat': 33, 
                                             'wpi': 43, 
                                             'silver': 42, 
                                             'cpi': 18, 
                                             'earn': 3, 
                                             'bop': 36, 
                                             'money-supply': 8, 
                                             'hog': 44, 
                                             'acq': 4, 
                                             'pet-chem': 39, 
                                             'grain': 1, 
                                             'retail': 29}, 
                                       orient='index',
                                       columns=['topic_id'])
                            
topic_indexes = topic_indexes.rename_axis('topic_name').reset_index()
topic_indexes = topic_indexes.set_index('topic_id').sort_index()
topic_indexes

,topic_name
topic_id,
0,cocoa
1,grain
2,veg-oil
3,earn
4,acq
5,wheat
6,copper
7,housing
8,money-supply


In [5]:
# Identify articles from training dataset with topic "orange".
np.where(y_train == 38)

(array([ 321, 1395, 1660, 2107, 2334, 2774, 2903, 3611, 3629, 3937, 4102,
        4506, 5213, 5894, 6591, 6718, 7757, 8059, 8975], dtype=int64),)

In [6]:
# Create reverse lookup dictionary for article reconstruction.
wordDict = {y:x for x, y in reuters.get_word_index().items()} 

# Reconstruct sample "orange" article.
print(' '.join([(wordDict.get(index-3)) for index in 
                filter(lambda i: (i >= 3), 
                       X_train[321])]))

there is no confirmation that brazil's major processors of frozen concentrated orange juice fcoj will raise export prices of the product to 1 375 dlrs per tonne from april 1 a spokesman for the brazilian association of citrus juice industries abrassuco said asked to comment on a report from new york that cutrale and citrosuco had sent telexes to customers informing of the price raise jose carlos goncalves said abrassuco was not aware of it all we know is that cacex has increased the dollar amount to translate fob price to ex dock new york price to 1 050 dlrs from 770 dlrs goncalves said citrosuco and cutrale officials were not available for comment reuter 3


In [7]:
# Look at topic distribution in training dataset.
pd.merge(pd.DataFrame(y_train, columns=['topic_id']), 
         topic_indexes, 
         on='topic_id')['topic_name'].value_counts(normalize=True)

earn               0.350590
acq                0.214986
money-fx           0.061456
grain              0.049878
crude              0.047651
trade              0.041527
interest           0.030728
ship               0.018815
money-supply       0.016477
sugar              0.014139
gnp                0.012469
coffee             0.011356
gold               0.011022
veg-oil            0.008461
oilseed            0.007793
cpi                0.007348
cocoa              0.006123
bop                0.005455
ipi                0.005344
reserves           0.005233
copper             0.005233
jobs               0.005010
alum               0.004676
livestock          0.004676
iron-steel         0.004565
nat-gas            0.004231
dlr                0.003785
rubber             0.003451
gas                0.003340
tin                0.002672
wpi                0.002561
cotton             0.002561
pet-chem           0.002561
carcass            0.002449
retail             0.002227
meal-feed          0

In [8]:
# Check that all classes are represented in both datasets.
set(y_train) == set(y_test)

True

In [9]:
# Check total number of words.
len(word_index.values())

30979

In [10]:
batch_size = 46
max_features = len(word_index.values()) + 1
maxlen = 200

print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print('Pad sequences (samples x time)')
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

print('Build model...')
model = Sequential()
model.add(Embedding(max_features, 128))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(len(set(y_train)), activation='softmax'))

8982 train sequences
2246 test sequences
Pad sequences (samples x time)
X_train shape: (8982, 200)
X_test shape: (2246, 200)
Build model...


In [11]:
# You should only run this cell once your model has been
# properly configured.
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print('Train...')
model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=1,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_train, y_train,
                            batch_size=batch_size,
                            verbose=False)
print('Train score:', score)
print('Train accuracy:', acc)

score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size,
                            verbose=False)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 8982 samples, validate on 2246 samples
8982/8982 [==============================] - 83s 9ms/sample - loss: 2.3513 - accuracy: 0.4059 - val_loss: 1.9899 - val_accuracy: 0.4960
Train score: 2.0117549815280906
Train accuracy: 0.48953462
Test score: 1.9898654584468525
Test accuracy: 0.49599287


In [12]:
# Define majority class from training dataset.
majority_class = scipy.stats.mode(y_train)[0][0]
majority_class

3

In [13]:
# Calculate accuracy of majority classifier for comparison.
acc = accuracy_score(y_train, np.full_like(y_train, majority_class))
print('Majority classifier train accuracy:', acc)

acc = accuracy_score(y_test, np.full_like(y_test, majority_class))
print('Majority classifier test accuracy:', acc)

Majority classifier train accuracy: 0.3505900690269428
Majority classifier test accuracy: 0.3664292074799644


## Sequence Data Question
#### *Describe the `pad_sequences` method used on the training dataset. What does it do? Why do you need it?*

> The `pad_sequences` method ([API documentation](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences)) used on the training and test datasets truncates inputs longer than the maximum length specified and prepends zeros to any inputs shorter than that length. This is necessary because the architecture of a neural network is designed to work with inputs of a set length, but our raw data is presumably of variable length.

## RNNs versus LSTMs
#### *What are the primary motivations behind using Long-ShortTerm Memory Cell unit over traditional Recurrent Neural Networks?*

> Traditional RNNs are prone to vanishing and exploding gradients, which can cause slow convergence or failure to converge. They also heavily weight the most recent inputs, which means that they don't model long-term dependencies well. LSTMs are designed to address both of these issues.

## RNN / LSTM Use Cases
#### *Name and Describe 3 Use Cases of LSTMs or RNNs and why they are suited to that use case*

> LSTMs - and RNNs generally - are useful for prediction and generation tasks where inputs occur in a meaningful and/or relevant sequential context, because of their ability to retain and use information about that context, i.e. past inputs. Examples include machine translation, text generation, music composition, handwriting and speech recognition, robot control, and many kinds of time series prediction.

<a id="p2"></a>
## Part 2- CNNs

### Find the Frog

Time to play "find the frog!" Use Keras and ResNet50 (pre-trained) to detect which of the following images contain frogs:

<img align="left" src="https://d3i6fh83elv35t.cloudfront.net/newshour/app/uploads/2017/03/GettyImages-654745934-1024x687.jpg" width=400>


In [ ]:
!pip install google_images_download

In [ ]:
from google_images_download import google_images_download

response = google_images_download.googleimagesdownload()
arguments = {"keywords": "lilly frog pond", "limit": 5, "print_urls": True}
absolute_image_paths = response.download(arguments)

At time of writing at least a few do, but since the Internet changes - it is possible your 5 won't. You can easily verify yourself, and (once you have working code) increase the number of images you pull to be more sure of getting a frog. Your goal is to validly run ResNet50 on the input images - don't worry about tuning or improving the model.

*Hint* - ResNet 50 doesn't just return "frog". The three labels it has for frogs are: `bullfrog, tree frog, tailed frog`

*Stretch goals* 
- Check for fish or other labels
- Create a matplotlib visualizations of the images and your prediction as the visualization label

In [ ]:
# You've got something to do in this cell. ;)

import numpy as np

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

def process_img_path(img_path):
  return image.load_img(img_path, target_size=(224, 224))

def img_contains_frog(img):
    """ Scans image for Frogs
    
    Should return a boolean (True/False) if a frog is in the image.
    
    Inputs:
    ---------
    img:  Precrossed image ready for prediction. The `process_img_path`             function should already be applied to the image. 
    
    Returns: 
    ---------
    frogs (boolean):  TRUE or FALSE - There are frogs in the image.
    
    """
    # Your Code Here
    # TODO - your code!
            
    return None

#### Stretch Goal: Displaying Predictions

In [ ]:
import matplotlib.pyplot as plt



<a id="p3"></a>
## Part 3 - Autoencoders

Describe a use case for an autoencoder given that an autoencoder tries to predict its own input. 

__*Your Answer:*__ 

<a id="p4"></a>
## Part 4 - More...

Answer the following questions, with a target audience of a fellow Data Scientist:

- What do you consider your strongest area, as a Data Scientist?
- What area of Data Science would you most like to learn more about, and why?
- Where do you think Data Science will be in 5 years?
- What are the threats posed by AI to our society?
- How do you think we can counteract those threats? 
- Do you think achieving General Artifical Intelligence is ever possible?

A few sentences per answer is fine - only elaborate if time allows.

## Congratulations! 

Thank you for your hard work, and congratulations! You've learned a lot, and you should proudly call yourself a Data Scientist.


In [ ]:
from IPython.display import HTML

HTML("""<iframe src="https://giphy.com/embed/26xivLqkv86uJzqWk" width="480" height="270" frameBorder="0" class="giphy-embed" allowFullScreen></iframe><p><a href="https://giphy.com/gifs/mumm-champagne-saber-26xivLqkv86uJzqWk">via GIPHY</a></p>""")